In [1]:
import pandas as pd

# enable the %%R magic
%load_ext rpy2.ipython

In [2]:
rep_score_correlation = pd.read_csv("/g/savitski/02_projects/99_EMBLcoll/Furlong/MPerino/MatteoPerino_2DTPP/Data_replicate_score_correlation_V1.csv")
rep_score_correlation.head()

,Unnamed: 0,gene_name,id,score,rep1,rep2,rep3
0,1,128UP,stage2,abundance.score,-0.041771,0.003306,0.130454
1,2,128UP,stage2,stability.score,0.107170,0.110215,-0.186837
2,3,128UP,stage3,abundance.score,-0.163658,-0.164838,-0.061904
3,4,128UP,stage3,stability.score,0.448462,0.146959,0.019360
4,5,128UP,stage3.2,abundance.score,-0.113644,-0.159900,-0.184114


In [6]:
rep_stability_correlation = rep_score_correlation[rep_score_correlation.score=="stability.score"]
rep_stability_correlation.loc[:,"rep2_1"] = rep_stability_correlation["rep2"].subtract(rep_stability_correlation["rep1"])
rep_stability_correlation.loc[:,"rep3_1"] = rep_stability_correlation["rep3"].subtract(rep_stability_correlation["rep1"])
rep_stability_correlation.loc[:,"rep3_2"] = rep_stability_correlation["rep3"].subtract(rep_stability_correlation["rep2"])
rep_stability_correlation.head()

,Unnamed: 0,gene_name,id,score,rep1,rep2,rep3,rep2_1,rep3_1,rep3_2
1,2,128UP,stage2,stability.score,0.107170,0.110215,-0.186837,0.003045,-0.294006,-0.297051
3,4,128UP,stage3,stability.score,0.448462,0.146959,0.019360,-0.301502,-0.429101,-0.127599
5,6,128UP,stage3.2,stability.score,0.275787,-0.028761,0.140691,-0.304548,-0.135095,0.169453
7,8,14,stage2,stability.score,-0.325422,-0.126074,1.011208,0.199348,1.336630,1.137282
9,10,14,stage3,stability.score,-0.544368,1.129489,0.046431,1.673857,0.590799,-1.083058


In [23]:
variable_stability = rep_stability_correlation[(rep_stability_correlation.rep2_1 > 4) | (rep_stability_correlation.rep2_1 < -4) |
                                               (rep_stability_correlation.rep3_1 > 4) | (rep_stability_correlation.rep3_1 < -4) |
                                               (rep_stability_correlation.rep3_2 > 4) | (rep_stability_correlation.rep3_2 < -4)]
print(variable_stability.shape)
variable_stability.head()


(238, 10)


,Unnamed: 0,gene_name,id,score,rep1,rep2,rep3,rep2_1,rep3_1,rep3_2
723,724,ARPC5,stage3,stability.score,0.420522,3.311257,-1.528736,2.890734,-1.949259,-4.839993
725,726,ARPC5,stage3.2,stability.score,1.527715,6.358456,-1.948874,4.830742,-3.476588,-8.307330
919,920,ATPSYNCF6,stage2,stability.score,-3.731967,-5.166830,3.372920,-1.434863,7.104888,8.539750
921,922,ATPSYNCF6,stage3,stability.score,-6.034136,0.412961,1.462502,6.447097,7.496638,1.049541
923,924,ATPSYNCF6,stage3.2,stability.score,-2.367674,5.514286,-1.975924,7.881960,0.391751,-7.490209


In [8]:
variable_stability_genes = variable_stability["gene_name"].drop_duplicates()
variable_stability_genes.shape
variable_stability_genes.head()

723         ARPC5
919     ATPSYNCF6
939       ATPSYNE
1011          B52
1017         BACC
Name: gene_name, dtype: object

In [44]:
variable_stability_long = variable_stability.drop(labels=["Unnamed: 0", "score","rep1","rep2","rep3"], axis="columns")
variable_stability_long = pd.melt(variable_stability_wide,id_vars=["gene_name","id",], var_name= "comparison", value_vars=["rep2_1","rep3_1","rep3_2"], value_name="zscore_change")
#variable_stability_wide = variable_stability_wide.pivot(columns=,values="gene_name")#["rep2_1","rep3_1","rep3_2"])
#variable_stability_wide.reset_index(drop=True, inplace=True)
variable_stability_long.head()                                                                      

,gene_name,id,comparison,zscore_change
0,ARPC5,stage3,rep2_1,2.890734
1,ARPC5,stage3.2,rep2_1,4.830742
2,ATPSYNCF6,stage2,rep2_1,-1.434863
3,ATPSYNCF6,stage3,rep2_1,6.447097
4,ATPSYNCF6,stage3.2,rep2_1,7.881960


In [37]:
background = rep_score_correlation["gene_name"].drop_duplicates()
print(background.shape)
background.head()

(3406,)


0     128UP
6        14
12    140UP
18       26
24      312
Name: gene_name, dtype: object

In [46]:
%%R
library(clusterProfiler)
library(org.Dm.eg.db)
library(dplyr)
library(ggplot2)
library(viridis)

/g/furlong/perino/tools/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/g/furlong/perino/tools/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Creating a generic function for ‘chol2inv’ from ‘base’ in package ‘Matrix’
    (from the saved implicit definition)

  warnings.warn(x, RRuntimeWarning)
/g/furlong/perino/tools/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Creating a generic function for ‘chol2inv’ from package ‘base’ in package ‘Matrix’

  warnings.warn(x, RRuntimeWarning)
/g/furlong/perino/tools/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Creating a generic function for ‘diag’ from package ‘base’ in package ‘Matrix’

  warnings.warn(x, RRuntimeWarning)
/g/furlong/perino/tools/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Creating a ge


Error: package or namespace load failed for ‘clusterProfiler’:
 unable to load R code in package ‘Matrix’


In [ ]:
%%R -i variable_stability_wide,background -w 5 -h 5 --units in -r 300
# import variable_stability_wide,background from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution



In [ ]:
%%R
clusters = list(names(Kc_stabilized_list), names(embryo_stabilized_list))
names(clusters) = c("Kc", "embryo")
clusters_GO <- compareCluster(geneCluster   = clusters, 
                              fun           = "enrichGO",
                              universe      = GO_data$Protein_ID,
                              OrgDb         = org.Dm.eg.db,
                              ont           = "ALL",
                              keyType       = "SYMBOL",
                              pAdjustMethod = "BH",
                              minGSSize     = 10,
                              maxGSSize     = 500,
                              pvalueCutoff  = 1,
                              qvalueCutoff  = 1)